In [1]:
import torchvision as vision
from torch.utils.data import DataLoader
from torchsummary import summary

from temp_dataset import TempData

In [2]:
# Use pretrained model for comparison
model = vision.models.wide_resnet50_2(pretrained=True)

summary(model, (3, 32, 32), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5            [-1, 128, 8, 8]           8,192
       BatchNorm2d-6            [-1, 128, 8, 8]             256
              ReLU-7            [-1, 128, 8, 8]               0
            Conv2d-8            [-1, 128, 8, 8]         147,456
       BatchNorm2d-9            [-1, 128, 8, 8]             256
             ReLU-10            [-1, 128, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]          32,768
      BatchNorm2d-12            [-1, 256, 8, 8]             512
           Conv2d-13            [-1, 256, 8, 8]          16,384
      BatchNorm2d-14            [-1, 25

In [3]:
model.eval()
dataset = TempData(file_n="/home/jaeha/Storage/Datasets/UTKFace/UTKFace/1_0_0_20161219140627985.jpg.chip.jpg")
dataloader = DataLoader(dataset=dataset, batch_size=1)

for batch in dataloader:
    res = model(batch)
    print(res)

tensor([[-2.2369e+02, -1.7273e+02, -1.9592e+02,  1.7746e+02, -1.6955e+02,
          4.4872e+01, -2.9754e+02, -9.3712e+01, -1.7969e+02,  1.0233e+02,
         -4.0376e+02,  9.9994e+01,  1.5128e+01, -6.7117e+01, -2.5554e+02,
         -1.0081e+02, -1.3889e+02,  8.8303e+01, -7.2255e+01,  1.0280e+02,
         -3.3216e+02, -1.2586e+02, -1.7541e+02, -3.9938e+01,  1.2470e+02,
         -2.7938e+02, -4.0264e+01, -1.5507e+02,  1.8256e+02, -3.2594e+02,
         -4.3484e+02,  2.8033e+02, -5.7094e+01,  1.0710e+02, -1.1726e+02,
         -1.7318e+02,  2.2750e+02, -9.4486e+01,  1.9217e+02,  1.0381e+02,
          2.4009e+02,  1.4940e+02,  3.3221e+02, -1.4077e+02, -1.1782e+01,
         -4.1541e+00,  3.4787e+02, -1.0890e+02, -2.4844e+02, -1.8687e+02,
         -1.3536e+02, -1.1695e+02,  5.2522e+02,  4.8720e+02,  1.5413e+02,
          6.7591e+02,  1.8653e+02, -2.7231e+02, -1.3218e+02,  2.3986e+02,
          8.1103e+02, -1.0260e+02, -2.6292e+01, -2.4505e+01,  1.4576e+02,
          1.2688e+02, -8.9247e+01, -2.